In [2]:
import time

from rfsoc_qsfp_offload.overlay import Overlay

print("Initializing RFSoC QSFP Offload Overlay")
ol = Overlay(ignore_version=True)
# Wait for overlay to initialize
time.sleep(5)

ol.cmac.mmio.write(0x107C, 0x3) # RSFEC_CONFIG_ENABLE
ol.cmac.mmio.write(0x1000, 0x7) # RSFEC_CONFIG_INDICATION_CORRECTION

ol.cmac.start()

board_ip = '192.168.4.99'
ol.netlayer.set_ip_address(board_ip, debug=True)

Initializing RFSoC QSFP Offload Overlay


{'HWaddr': '00:0a:35:02:9d:63',
 'inet addr': '192.168.4.99',
 'gateway addr': '192.168.4.1',
 'Mask': '255.255.255.0'}

In [3]:
client_ip = '192.168.4.1'
ol.netlayer.sockets[0] = (client_ip, 60133, 60133, True)

ol.netlayer.populate_socket_table()

ol.source_select(1) # 0 - DMA | 1 - RF-ADC

## Configure ADC

In [4]:
ADC_TILE = 2       # ADC Tile 226
ADC_BLOCK = 0       # ADC Block 0
#ADC_SAMPLE_FREQUENCY = 1000  # MSps
#ADC_SAMPLE_FREQUENCY = 1966.08# MSps
ADC_SAMPLE_FREQUENCY = 1228.8 # MSps
ADC_PLL_FREQUENCY    = 491.52   # MHz
ADC_FC = 0e6 # Centering around middle of sample rate

ol.initialise_adc(tile=ADC_TILE,
                  block=ADC_BLOCK,
                  pll_freq=ADC_PLL_FREQUENCY,
                  fs=ADC_SAMPLE_FREQUENCY,
                  fc=ADC_FC)

# Decimate by (16,5) 76.8MHz
ol.set_decimation(tile=ADC_TILE,block=ADC_BLOCK,sample_rate=307200000.0)
# Decimate by (8,4)
#ol.set_decimation(tile=ADC_TILE,block=ADC_BLOCK,sample_rate=614400000.0) 
# Decimate by (4,3)
#ol.set_decimation(tile=ADC_TILE,block=ADC_BLOCK,sample_rate=1228800000.0) 
# Decimate by (4,3)
#ol.set_decimation(tile=ADC_TILE,block=ADC_BLOCK,sample_rate=2457600000.0) 

# Set packet size
ol.packet_generator.packetsize = 128 # 128 * 64 bytes = 8192 bytes to be sent
ol.packet_generator.enable()

['ClockSource', 'DumpRegs', 'DynamicPLLConfig', 'FIFOStatus', 'FabClkOutDiv', 'GetFIFOStatusObs', 'PLLConfig', 'PLLLockStatus', 'Reset', 'SetupFIFO', 'SetupFIFOBoth', 'SetupFIFOObs', 'ShutDown', 'StartUp', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_call_function', '_call_function_implicit', '_index', '_parent', '_type', 'blocks']


## Configure DAC

In [5]:
DAC_TILE = 0       # DAC Tile 228
DAC_BLOCK = 0       # DAC Block 0
#DAC_SAMPLE_FREQUENCY = 1966.08  # MSps
DAC_SAMPLE_FREQUENCY = 1228.8  # MSps
DAC_PLL_FREQUENCY = 491.52   # MHz
DAC_FC = 0e6

ol.initialise_dac(tile=DAC_TILE,
                  block=DAC_BLOCK,
                  pll_freq=DAC_PLL_FREQUENCY,
                  fs=DAC_SAMPLE_FREQUENCY,
                  fc=DAC_FC
                 )


In [6]:
from rfsoc_qsfp_offload import signal_generator
import numpy as np
from pynq import allocate


sin_data = signal_generator.sine(f=10.2e6, fs=DAC_SAMPLE_FREQUENCY*1e6/4)
#sin_data = signal_generator.sine(f=81.6e6, fs=DAC_SAMPLE_FREQUENCY*1e6/4)
sin_data = signal_generator.convert_to_int16(sin_data)

zero_data = np.zeros(sin_data.size, np.int16)
interleaved = np.empty((sin_data.size + zero_data.size,), dtype=sin_data.dtype)
interleaved[0::2] = sin_data
interleaved[1::2] = zero_data

# Send signal to DAC over DMA
tx_buffer = allocate(shape=(interleaved.size,), dtype=np.int16)
tx_buffer[:] = interleaved
ol.axi_dma_dac.sendchannel.transfer(tx_buffer, cyclic=True)

#ol.rfdc.dac_tiles[0].blocks[0].MixerSettings['Freq'] = 1

In [7]:
import plotly.graph_objs as go
go.FigureWidget(go.Scatter(y=sin_data[1:8192]))

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '3aa8650a-a2ee-4d94-97cd-80cd9868616e',
 …

In [13]:
ol.rfdc.dac_tiles[0].blocks[0].MixerSettings['Freq'] = .1 # MHz

### Shutdown the Packet Generator and DMA

Make sure to stop the packet generator and DMA by running the cell below

In [13]:
ol.packet_generator.disable()
ol.axi_dma_dac.sendchannel.stop()